##### Copyright 2025.

# Gemma 2 Function Calling with LangChain and Groq

This tutorial demonstrates how to perform **function calling** using the `gemma-2-9b-it` model hosted on [Groq Cloud](https://console.groq.com/) with **LangChain integration**.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/your-repo/gemma-groq-function-calling-langchain.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

You will need:

- A [Groq API key](https://console.groq.com/keys)
- Python 3.10+
- Colab secrets enabled to securely access the key.

In [2]:
!pip install --quiet groq langchain langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 1.8 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata

# Load your Groq API key from Colab secrets
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

## Define External Tools (Functions)

We define a sample function that fetches weather information.

In [27]:
from langchain.tools import Tool
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent
import re

# Define a single-input weather tool
def get_current_weather_tool(input_str: str) -> str:
    """Get the current weather in a location."""
    # Naive parsing: look for location and optional unit
    match = re.search(r"in (\w+)", input_str.lower())
    unit_match = re.search(r"(celsius|fahrenheit)", input_str.lower())

    location = match.group(1) if match else "unknown"
    unit = unit_match.group(1) if unit_match else "fahrenheit"
    return f"Final Answer: The weather in {location.title()} is 72° {unit.title()} and sunny."




## Initialize LangChain Agent with Gemma 2 on Groq

We'll use `gemma-2-9b-it` via LangChain and bind our tool.

In [28]:
get_current_weather = Tool(
    name="get_current_weather",
    func=get_current_weather_tool,
    description="Get the current weather by passing a string like 'What's the weather in Paris in Celsius?'",
    return_direct=True
)

# Initialize LLM
llm = ChatGroq(model="gemma-2-9b-it", temperature=0)

# Initialize the agent (no need to bind tools for this setup)
agent = initialize_agent(
    tools=[get_current_weather],
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)



## Run Example Prompt

We now run a natural language prompt that should trigger function calling.

In [29]:
# Example usage
response = agent.run("What's the weather in Tokyo in Fahrenheit?")
print(response)



> Entering new AgentExecutor chain...
Thought: I need to get the current weather in Tokyo, and the user wants the temperature in Fahrenheit. I should use the get_current_weather function to get the current weather.

Action: get_current_weather
Action Input: What's the weather in Tokyo in Fahrenheit?
Observation: Final Answer: The weather in Tokyo is 72° Fahrenheit and sunny.


> Finished chain.
Final Answer: The weather in Tokyo is 72° Fahrenheit and sunny.


## Conclusion

In this notebook, you learned how to:

- Connect Gemma 2 (`gemma-2-9b-it`) via Groq using LangChain
- Define and bind external tools (functions)
- Use natural prompts to trigger function calls via LangChain Agents

This is a powerful building block for integrating APIs and logic into LLM workflows.